In [22]:
import numpy as np
import pandas as pd

from scipy import stats

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sktime.transformations.series.date import DateTimeFeatures
from sktime.transformations.series.fourier import FourierFeatures


import plotly.graph_objects as go
import matplotlib
matplotlib.use('agg')

In [2]:
class LogTransformation:
    @staticmethod
    def transform(x):
        xt = np.sign(x) * np.log(np.abs(x) + 1)

        return xt

    @staticmethod
    def inverse_transform(xt):
        x = np.sign(xt) * (np.exp(np.abs(xt)) - 1)

        return x


In [21]:
fig = go.Figure()
arg = np.linspace(-1000, 1000, 200)
fig.add_trace(go.Scatter(x=arg, y=LogTransformation.transform(arg), name="transform"))

fig.show()

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if len(data.shape) == 1 else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("var%d(t-%d)" % (j + 1, i)) for j in range(n_vars)]

    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [("var%d(t)" % (j + 1)) for j in range(n_vars)]
        else:
            names += [("var%d(t+%d)" % (j + 1, i)) for j in range(n_vars)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names

    if dropnan:
        agg.dropna(inplace=True)

    return agg

series = pd.read_csv(
    "../assets/datasets/time_series_solar.csv",
    parse_dates=["Datetime"],
    index_col="Datetime",
)["Incoming Solar"]

# Resample the data to daily frequency
series = series.resample("D").sum()
series.tail()


Datetime
2013-09-26    1034.2
2013-09-27    2182.3
2013-09-28    3384.5
2013-09-29     478.2
2013-09-30    2554.8
Freq: D, Name: Incoming Solar, dtype: float64

In [5]:
train, test = train_test_split(series, test_size=0.2, shuffle=False)

# Log
train_log = LogTransformation.transform(train)
test_log = LogTransformation.transform(test)

# Box-Cox
train_bc, bc_lambda = stats.boxcox(train)
train_bc = pd.Series(train_bc, index=train.index)

test_bc = stats.boxcox(test, lmbda=bc_lambda)
test_bc = pd.Series(test_bc, index=test.index)

scaler = MinMaxScaler(feature_range=(-1, 1))

train_norm = scaler.fit_transform(train_bc.values.reshape(-1, 1))
test_norm = scaler.transform(test_bc.values.reshape(-1, 1))

train_df = series_to_supervised(train_norm, 3).values
test_df = series_to_supervised(test_norm, 3).values

In [6]:
X_train, y_train = train_df[:, :-1], train_df[:, -1]
X_test, y_test = test_df[:, :-1], test_df[:, -1]

X_train = torch.from_numpy(X_train).type(torch.Tensor)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor).view(-1)
y_test = torch.from_numpy(y_test).type(torch.Tensor).view(-1)

X_train = X_train.view([X_train.shape[0], X_train.shape[1], 1])
X_test = X_test.view([X_test.shape[0], X_test.shape[1], 1])

In [7]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

In [9]:
model = LSTM(input_dim=1, hidden_dim=32, output_dim=1, num_layers=2)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(-1)
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

model.eval()
y_pred = model(X_test).reshape(-1)

y_pred_np = y_pred.detach().numpy().reshape(-1, 1)
y_pred_denorm = scaler.inverse_transform(y_pred_np).flatten()

y_pred_orig = LogTransformation.inverse_transform(y_pred_denorm)

print(model)

Epoch: 0, Loss: 0.25972941517829895
Epoch: 10, Loss: 0.2493474930524826
Epoch: 20, Loss: 0.2354065179824829
Epoch: 30, Loss: 0.21062248945236206
Epoch: 40, Loss: 0.16634516417980194
Epoch: 50, Loss: 0.11073876917362213
Epoch: 60, Loss: 0.10527494549751282
Epoch: 70, Loss: 0.09918505698442459
Epoch: 80, Loss: 0.09926200658082962
Epoch: 90, Loss: 0.09759937971830368
Epoch: 100, Loss: 0.0969504714012146
Epoch: 110, Loss: 0.09600860625505447
Epoch: 120, Loss: 0.09513276815414429
Epoch: 130, Loss: 0.09418144822120667
Epoch: 140, Loss: 0.09316795319318771
Epoch: 150, Loss: 0.0921064242720604
Epoch: 160, Loss: 0.09100371599197388
Epoch: 170, Loss: 0.08989153057336807
Epoch: 180, Loss: 0.0888107642531395
Epoch: 190, Loss: 0.08781436830759048
LSTM(
  (lstm): LSTM(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


/tmp/ipykernel_382668/3352635587.py:10: RuntimeWarning: overflow encountered in exp
  x = np.sign(xt) * (np.exp(np.abs(xt)) - 1)


In [14]:
y_pred_orig.shape, train.shape, test.shape

((436,), (1753,), (439,))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index[:train.shape[0]], y=train, name="Incoming Solar"))
fig.add_trace(go.Scatter(x=series.index[train.shape[0]:], y=test, name="Actual"))
fig.add_trace(go.Scatter(x=series.index[train.shape[0]:], y=y_pred_orig, name="Prediction"))

fig.show()

In [25]:
series = pd.read_csv(
    "../assets/datasets/time_series_solar.csv",
    parse_dates=["Datetime"],
    index_col="Datetime",
)["Incoming Solar"]

# Resample the data to daily frequency
series = series.resample("D").sum()

# train test split
train, test = train_test_split(series, test_size=0.2, shuffle=False)

scaler = MinMaxScaler(feature_range=(-1, 1))

train_norm = scaler.fit_transform(train.values.reshape(-1, 1)).flatten()
train_norm = pd.Series(train_norm, index=train.index)

test_norm = scaler.transform(test.values.reshape(-1, 1)).flatten()
test_norm = pd.Series(test_norm, index=test.index)

train_df = series_to_supervised(train_norm, 3)
test_df = series_to_supervised(test_norm, 3)

In [28]:
### Seasonal dummies

date_features = DateTimeFeatures(
    ts_freq="D", keep_original_columns=False, feature_scope="efficient"
)

train_dates = date_features.fit_transform(train_df.iloc[:, -1])
train_dates.tail()


,year,quarter_of_year,month_of_year,week_of_year,day_of_year,day_of_month,day_of_week
Datetime,,,,,,,
2012-07-14,2012,3,7,28,196,14,5
2012-07-15,2012,3,7,28,197,15,6
2012-07-16,2012,3,7,29,198,16,0
2012-07-17,2012,3,7,29,199,17,1
2012-07-18,2012,3,7,29,200,18,2


In [30]:
train_dates = train_dates[["month_of_year", "day_of_week"]]

encoder = OneHotEncoder(drop="first", sparse_output=False)
encoded_feats = encoder.fit_transform(train_dates)

train_dummies = pd.DataFrame(
    data=encoded_feats, 
    columns=encoder.get_feature_names_out(),
    index=train_dates.index, 
    dtype=int
)

train_dummies.tail()

,month_of_year_2,month_of_year_3,month_of_year_4,month_of_year_5,month_of_year_6,month_of_year_7,month_of_year_8,month_of_year_9,month_of_year_10,month_of_year_11,month_of_year_12,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
Datetime,,,,,,,,,,,,,,,,,
2012-07-14,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2012-07-15,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2012-07-16,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2012-07-17,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2012-07-18,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [32]:

test_dates = date_features.transform(test_df.iloc[:, -1])
test_dates = test_dates[["month_of_year", "day_of_week"]]

test_encoded_feats = encoder.transform(test_dates)

test_dummies = pd.DataFrame(
    test_encoded_feats, columns=encoder.get_feature_names_out(), dtype=int
)
test_dummies.tail()

,month_of_year_2,month_of_year_3,month_of_year_4,month_of_year_5,month_of_year_6,month_of_year_7,month_of_year_8,month_of_year_9,month_of_year_10,month_of_year_11,month_of_year_12,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
431,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
432,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
433,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
434,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
435,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [33]:
# Same for Fourier series


fourier = FourierFeatures(
    sp_list=[365.25], fourier_terms_list=[2], keep_original_columns=False
)

train_fourier = fourier.fit_transform(train_df.iloc[:, -1])
test_fourier = fourier.transform(test_df.iloc[:, -1])
train_fourier[:-5]

/home/volody/code/study-py/ts-pytorch/.venv/lib/python3.11/site-packages/sktime/transformations/base.py:475: UserWarning:

Using frequency from index: <Day>, which does not match the frequency given:None.



,sin_365.25_1,cos_365.25_1,sin_365.25_2,cos_365.25_2
Datetime,,,,
2007-10-04,0.000000,1.000000,0.000000,1.000000
2007-10-05,0.017202,0.999852,0.034398,0.999408
2007-10-06,0.034398,0.999408,0.068755,0.997634
2007-10-07,0.051584,0.998669,0.103031,0.994678
2007-10-08,0.068755,0.997634,0.137185,0.990545
...,...,...,...,...
2012-07-09,-0.996210,0.086977,-0.173295,-0.984870
2012-07-10,-0.994567,0.104101,-0.207070,-0.978326
2012-07-11,-0.992629,0.121193,-0.240600,-0.970624


In [35]:
X_train.shape, train_dummies.shape, train_fourier.shape

(torch.Size([1750, 3, 1]), (1750, 17), (1750, 4))

In [34]:
### appending
X_train = np.hstack([X_train, train_dummies, train_fourier])
X_test = np.hstack([X_test, test_dummies, test_fourier])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)